## 네이버 블로그에서 본문 수집

### xlsx 파일 읽기

In [ ]:
# 관련 라이브러리를 호출합니다.
import os
import pandas as pd

In [ ]:
# 현재 작업경로를 확인합니다.
os.getcwd()

In [ ]:
# data 폴더로 작업경로를 변경합니다.
os.chdir(path = '../data')

In [ ]:
# 현재 작업경로에 저장된 폴더명과 파일명을 출력합니다.
os.listdir()

In [ ]:
# xlsx 파일을 읽고 데이터프레임을 생성합니다.
blog = pd.read_excel(io = 'Naver Blog Links.xlsx')

In [ ]:
# xlsx 파일의 처음 5행을 출력합니다.
blog.head()

### HTTP 요청 실행

- \<iframe\> 태그에는 네이버 블로그 링크가 아래 형태로 되어 있습니다.<br>
    https://blog.naver.com/PostView.nhn?blogId={blogId}&logNo={logNo}

In [ ]:
# 첫 번째 행의 blogId와 logNo를 각각 생성합니다.
i = 0
blogId = blog['blogId'][i]
logNo = blog['logNo'][i]

In [ ]:
# 요청 URL을 조립합니다.
URL = f'https://blog.naver.com/PostView.nhn?blogId={blogId}&logNo={logNo}'
print(URL)

In [ ]:
# 관련 라이브러리를 호출합니다.
import requests

In [ ]:
# HTTP 요청을 실행합니다. 
res = requests.get(url = URL)

### HTTP 응답 확인

In [ ]:
# HTTP 응답 상태코드를 확인합니다.
res.status_code

In [ ]:
# HTTP 응답 헤더를 확인합니다. [참고] Content-Type이 'html'입니다.
res.headers

In [ ]:
# HTTP 요청 URL을 확인합니다.
res.url

In [ ]:
# HTTP 응답 바디(str)의 일부를 출력합니다.
res.text[:1000]

### HTML 파싱(Parsing)

In [ ]:
# 관련 라이브러리를 호출합니다.
from bs4 import BeautifulSoup as bts

In [ ]:
# 문자열(str)을 bs4.BeautifulSoup 자료형으로 변환합니다.
soup = bts(markup = res.text, features = 'html.parser')

### 블로그 본문 수집

In [ ]:
# 블로그 본문을 포함하는 CSS Selector를 설정합니다.
css = '.se-main-container, .__se_component_area, #postViewArea'

In [ ]:
# 블로그 본문을 포함하는 HTML 요소의 텍스트를 body에 할당합니다.
body = soup.select(selector = css)[0].text
body

In [ ]:
# 관련 라이브러리를 호출합니다.
import re

In [ ]:
# 정규표현식으로 한글, 숫자, 마침표 및 공백을 제외하고 모두 삭제합니다.
pat = re.compile(r'[^가-힣ㄱ-ㅣ0-9. ]')
text = re.sub(pattern = pat, repl = '', string = body)
text

### for 반복문 실행

In [ ]:
# 관련 라이브러리를 호출합니다.
import tqdm, time

In [ ]:
# blog의 행 길이를 설정합니다.(반복문 실행 횟수)
n = blog.shape[0]
print(n)

In [ ]:
# 블로그 본문을 저장할 contents 열을 생성하고 빈 문자열을 할당합니다.
blog['contents'] = ''

In [ ]:
# for 반복문을 실행합니다.
for i in tqdm.tqdm_notebook(range(n)):
    
    # 각 행의 blogId와 logNo를 각각 생성합니다.
    blogId = blog['blogId'][i]
    logNo = blog['logNo'][i]
    
    # URL을 조립합니다.
    URL = f'https://blog.naver.com/PostView.nhn?blogId={blogId}&logNo={logNo}'
    
    # HTTP 요청을 실행합니다. 
    res = requests.get(url = URL)
    
    # 문자열(str)을 bs4.BeautifulSoup 자료형으로 변환합니다.
    soup = bts(markup = res.text, features = 'html.parser')
    
    # 블로그 본문이 없으면 에러가 발생하므로 예외처리합니다.
    try:
            
        # 블로그 본문을 포함하는 HTML 요소의 텍스트를 body에 할당합니다.
        css = '.se-main-container, .__se_component_area, #postViewArea'
        body = soup.select(selector = css)[0].text
        
        # 정규표현식으로 한글, 숫자, 마침표 및 공백을 제외하고 모두 삭제합니다.    
        pat = re.compile('[^가-힣ㄱ-ㅣ0-9. ]')
        text = re.sub(pattern = pat, repl = '', string = body)
        
    except:
        
        print(f'{i}번 블로그 본문이 없습니다!')
        text = ''
    
    # contents 열에 text를 추가합니다.
    blog.loc[i, 'contents'] = text
    
    # 1초간 멈춥니다.
    time.sleep(1)
    

In [ ]:
# blog의 처음 5행을 출력합니다.
blog.head()

In [ ]:
# blog의 정보를 확인합니다.
blog.info()

In [ ]:
# logNo를 문자열로 변환합니다.
blog['logNo'] = blog['logNo'].astype('str')

### xlsx 파일로 저장

In [ ]:
# xlsx 파일명을 설정합니다.
fileName = 'Naver Blog Contents.xlsx'

In [ ]:
# xlsx 파일로 저장합니다.
blog.to_excel(excel_writer = fileName, index = None)

# End of Document